In [1]:
import polars as pl
import pandas as pd
import random
import json
import datetime
import glob
import os
import io

In [2]:
current = datetime.datetime.now().strftime("%Y-%m-%d-%H")

In [5]:
list_of_files = glob.glob('data_municipios\*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
latest_file

data_municipios\20220501


In [6]:
df = pl.read_csv(f"{latest_file}/data.csv")

In [16]:
df_num_mun = df.groupby("Cve_Ent").agg(pl.struct("Cve_Mun").n_unique().alias('num_mun'))

In [17]:
dict_est_mun = {}

for a,b in zip(df_num_mun["Cve_Ent"],df_num_mun["num_mun"]):
    dict_est_mun[a] = b

In [52]:
list_mun_clima = []

In [53]:
for key, value in dict_est_mun.items():
    for mun in range(0, value):
        clima_dict = {}
        clima_dict["cc"] = random.uniform(89,91)
        clima_dict["ides"] = key
        clima_dict["idmun"] = mun
        clima_dict["dirvieng"] = random.randint(4,10)
        clima_dict["prec"] = random.uniform(0.50,1.50) 
        clima_dict["probprec"] = random.uniform(0,100)
        clima_dict["raf"] = random.uniform(14,30)
        clima_dict["tmax"] = random.uniform(15,40)
        clima_dict["tmin"] = random.uniform(0,15)
        clima_dict["velvien"] = random.uniform(0,100)
        clima_dict["probprec"] = random.uniform(0,15)
            
    
        list_mun_clima.append(clima_dict)

In [54]:
mun_clima_json = json.dumps(list_mun_clima)

In [35]:
with open(f"json_clima_municipios/{today}.json", 'w') as outfile:
    outfile.write(jsonStr)

In [55]:
mun_clima_io = io.StringIO(mun_clima_json)

In [56]:
df_mun_clima = pl.read_json(mun_clima_io)

In [41]:
df_mun_clima.write_csv(f"csv_clima_municipios/{current}.csv")

In [57]:
df_clima_current = df_mun_clima.clone()

In [58]:
list_of_files = glob.glob('csv_clima_municipios\*')
latest_file = max(list_of_files, key=os.path.getctime)
latest_file

'csv_clima_municipios\\2023-04-23-00.csv'

In [59]:
#Previous
df_clima_prev = pl.read_csv(latest_file)

#Current
df_clima_curr = df_clima_current.clone()

In [63]:
df_clima = df_clima_prev.join(df_clima_curr, on = ["ides","idmun"])

In [66]:
left_cols = ["dirvieng","prec","probprec","raf","tmax","tmin","velvien"]
right_cols = ["dirvieng_right","prec_right","probprec_right","raf_right","tmax_right","tmin_right","velvien_right"]

In [67]:
new_cols = ["ides","idmun"]
for l,r in zip(left_cols,right_cols):
    name = f"{l}_t"
    new_cols.append(name)
    df_clima = df_clima.with_columns((pl.col(l) + pl.col(r)).alias(name))

In [69]:
df_clima_avg = df_clima[new_cols]

In [75]:
df_clima_avg.columns = list(map(lambda x: x.replace("_t", ""), df_clima_avg.columns))

In [76]:
list_of_files = glob.glob('data_municipios/*')
latest_file = max(list_of_files, key=os.path.getctime)

In [78]:
df_mun= pl.read_csv(f"{latest_file}/data.csv")

In [79]:
df_clima_mun_avg = df_mun.join(df_clima_avg,left_on=['Cve_Ent','Cve_Mun'],right_on=['ides','idmun'],how='left')

In [82]:
df_clima_mun_avg.write_csv(f"clima_municipios_avg/{current}.csv")
df_clima_mun_avg.write_csv(f"clima_municipios_avg/current.csv")